Installing the required packages


In [1]:
!python --version




Python 3.9.18


In [2]:
import tensorflow as tf 
import os
#test

In [3]:
os.path.join('images',"birds")

'images\\birds'

In [4]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
gpu = tf.config.experimental.list_physical_devices("GPU")

In [6]:
print(gpu)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
import tensorflow as tf 
print(tf.__version__)

print('1: ', tf.config.list_physical_devices('GPU'))
print('2: ', tf.test.is_built_with_cuda)
print('3: ', tf.test.gpu_device_name())
print('4: ', tf.config.get_visible_devices())

2.10.1
1:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2:  <function is_built_with_cuda at 0x000001C2786CCCA0>
3:  /device:GPU:0
4:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2. Remove unsupported images and image formats

In [8]:
import cv2
import imghdr

In [9]:
train_dir = os.path.join("images","train")
image_exts = ["jpg","jpeg","png"]

In [10]:


# for image_dir in os.listdir(train_dir):
#     for image in os.listdir(os.path.join(train_dir, image_dir)):
#         image_path = os.path.join(train_dir, image_dir,image)
#         try:
#             img = cv2.imread(image_path)
#             ext = imghdr.what(image_path)
#             if ext not in image_exts:
#                 print('Image is not in ext list {}'.format(image_path))
#                 os.remove(image_path)
#         except Exception as e:
#             print("Issue with image {}".format(image_path))

            


3. Load Data

In [18]:
import numpy as np 
import matplotlib


ImportError: cannot import name 'Image' from 'PIL' (c:\Users\reach\miniconda3\lib\site-packages\PIL\__init__.py)